In [1]:
import tensorflow as tf
import cv2 as cv
import numpy as np
import matplotlib
import os
import random
#from PIL import Image
#%matplotlib inline
tf.logging.set_verbosity(tf.logging.INFO)
#tf.enable_eager_execution()
# from tensorflow.python import debug as tf_debug



In [2]:
# Loading Data
imgpath = 'data/images'
annpath = 'data/annotations/list.txt'


In [3]:

def loadAnnotations(annpath, imgpath):
    lbl = dict()
    img = dict()

    for i in os.listdir(imgpath):
        i = i[0:-4]
        img[i] = True

    with open(annpath,'r') as f:
        lines = f.readlines()
        i = 0
        for line in lines:
            line = line.rstrip('\n')
            words = (line.split(' '))
            if words[0][0] == '#':
                continue
            try:
                temp = img[words[0]]
            except:
                continue
            lbl[words[0]] = words[2]
            i = i+1
    
    return lbl

def label_lookup(x):
    narr = x.numpy()
    narr = narr.decode()
    try:
        key = int(anns[narr])
        return key
    except:
        raise KeyError("Error")

In [4]:
def parse_data(files, anns):

    files_e = tf.expand_dims(files, axis = 0)
    temp = tf.string_split(files_e, '.')
    label = tf.py_function(func = label_lookup, inp=[temp.values[0]], Tout = tf.int32)
    image_string = tf.read_file(imgpath +'/'+files)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.resize(image, (200,200), align_corners=True)
    image = tf.reshape(image,[200,200,3])
    image = tf.cast(image,tf.float32)
    image = image/255.0
    return (dict({'image':image}), label-1)#, files)


def train_input_fn(filenames,anns, epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(filenames).shuffle(True)
    dataset = dataset.map(lambda x: parse_data(x,anns))
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)
    iterator = dataset.make_one_shot_iterator()
    # return iterator
    return iterator.get_next()

def split_test_train(filenames, train_ratio):
    num = len(filenames)
    random.shuffle(filenames)
    train_num = int(num*train_ratio)
    test_num = num - train_num
    train = filenames[0:train_num-1]
    test = filenames[train_num-1:num-1]
    return train, test

def evaluate_input_fn(filenames, anns, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.map(lambda x: parse_data(x,anns)).batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

    


In [6]:

anns = loadAnnotations(annpath, imgpath)
filenames = [m + '.jpg' for m in anns.keys()]
[train_set, test_set] = split_test_train(filenames[:], 0.8)

dataset_l = len(filenames)

In [7]:

def model(features, labels, mode):
    # Input Layer
    input_layer = tf.reshape(features["image"], [-1, 200, 200, 3])
  
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu   
        )

    # Pooling layer #2 
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Convolutional layer #3
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu
        )

    # Pooling layer #3
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

    # Convolutional layer #4
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu
        )
    
    # Pooling layer #4
    pool4 = tf.layers.max_pooling2d(inputs=conv4 , pool_size=[2, 2], strides=2)

    # Dense Layer
    pool4_flat = tf.reshape(pool4, [-1, 12 * 12 * 64])
    dense = tf.layers.dense(inputs=pool4_flat, units=1024, activation=tf.nn.relu)
    logits = tf.layers.dense(inputs=dense, units=2)
    
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)



    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [8]:
classifier = tf.estimator.Estimator(
    model_fn=model, model_dir="./models")
    
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb9f5f274e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
   
classifier.train(lambda:train_input_fn(train_set,anns,10,32),
    hooks=[logging_hook])

# #%%
# with tf.Session() as sess:
    
#     sess.run(tf.global_variables_initializer())
#     it = train_input_fn(filenames,anns,1,1)
#     for i in range(1,1000000,1):
#         try:
#             b = it.get_next()
#             a = tf.shape((b[0]['image']))
#             bul = sess.run(a) == [1,200,200,3]
#             if bul.any() == False:
#                 print('Error '+sess.run(b[2]))
#         except tf.errors.OutOfRangeError:
#                 break

Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./models/model.ckpt-4674
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4674 into ./models/model.ckpt.
INFO:tensorflow:probabilities = [[0.0448036  0.9551964 ]
 [0.01845758 0.9815424 ]
 [0.01882716 0.98117286]
 [0.03712384 0.96287614]
 [0.03814872 0.9618513 ]
 [0.02

In [10]:
acc = classifier.evaluate(lambda:evaluate_input_fn(train_set,anns,128))
    


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-31T11:31:54Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/model.ckpt-4676
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-31-11:35:55
INFO:tensorflow:Saving dict for global step 4676: accuracy = 0.68289363, global_step = 4676, loss = 0.9168865
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4676: ./models/model.ckpt-4676
